In [ ]:
import altair as alt
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl


In [ ]:
def generate_all_combos(crashes: pl.DataFrame, group_by_cols):
    """
    Some groups have no crashes. This make sures these groups still have a row of 0s.
    """
    # create a single row df with each col being a list of the unique values of that col
    cross_product_df = crashes.select(pl.col(group_by_cols).unique().implode())
    # create a cross product of the unique values of these cols
    for col in group_by_cols:
        # explode explodes the lists in each of these cols back out to different rows
        cross_product_df = cross_product_df.explode(col)
    return cross_product_df.join(
        crashes, how="left", on=group_by_cols, coalesce=True
    ).with_columns(
        pl.col("area_sqmi").max().over("analysis_neighborhood"),
        pl.col("PEDESTRIAN_ACCIDENT", "BICYCLE_ACCIDENT", "ALL_ACCIDENT").fill_null(0),
    )


def group_crashes(crashes: pl.DataFrame, time_col: str):
    group_by_cols = [time_col, "analysis_neighborhood", "severity"]
    crashes_grouped = generate_all_combos(
        crashes.group_by(group_by_cols).agg(
            # area_sqmi dependent on the 'analysis_neighborhood' col, but can't use
            # group by directly, as it's a float and so the values are not exactly equal
            pl.first("area_sqmi"),
            pl.sum("PEDESTRIAN_ACCIDENT", "BICYCLE_ACCIDENT"),
            ALL_ACCIDENT=pl.len(),
        ),
        group_by_cols,
    )
    # also calculate number of injury only + fatal crashes
    return normalize_by_area(
        pl.concat(
            [
                crashes_grouped,
                crashes_grouped.group_by(time_col, "analysis_neighborhood").agg(
                    pl.lit("combined").alias("severity"),
                    pl.first("area_sqmi"),
                    pl.sum(
                        "PEDESTRIAN_ACCIDENT",
                        "BICYCLE_ACCIDENT",
                        "ALL_ACCIDENT",
                    ),
                ),
            ]
        ).sort(group_by_cols)
    )


def sum_ped_and_bike(crashes_grouped):
    return crashes_grouped.with_columns(
        PED_AND_BIKE_ACCIDENT=pl.sum_horizontal(
            "PEDESTRIAN_ACCIDENT", "BICYCLE_ACCIDENT"
        ),
        PED_AND_BIKE_ACCIDENT_per_sqmi=pl.sum_horizontal(
            "PEDESTRIAN_ACCIDENT_per_sqmi", "BICYCLE_ACCIDENT_per_sqmi"
        ),
    )


def normalize_by_area(crashes_grouped: pl.DataFrame):
    # normalizing by area rather than road-miles, because the road centrelines
    # GIS file sometimes has 2 'roads' per segment of arterial (e.g. north/southbound
    # are separate for Van Ness). I guess we can just count one-way-road-miles. But eh
    # area is good enough
    return crashes_grouped.with_columns(
        PEDESTRIAN_ACCIDENT_per_sqmi=pl.col("PEDESTRIAN_ACCIDENT")
        / pl.col("area_sqmi"),
        BICYCLE_ACCIDENT_per_sqmi=pl.col("BICYCLE_ACCIDENT") / pl.col("area_sqmi"),
        ALL_ACCIDENT_per_sqmi=pl.col("ALL_ACCIDENT") / pl.col("area_sqmi"),
    )


def calculate_covid_change(crashes_grouped: pl.DataFrame, geog_col):
    return (
        crashes_grouped.filter(pl.col("covid_period").is_in(["2018-2019", "2022-2023"]))
        .sort(geog_col, "severity", "covid_period")
        .with_columns(
            # calculate % change with previous row
            (
                pl.col(
                    # "PEDESTRIAN_ACCIDENT",
                    # "BICYCLE_ACCIDENT",
                    # "ALL_ACCIDENT",
                    "PED_AND_BIKE_ACCIDENT"
                ).pct_change()
                * 100  # since pct_change gives it in fraction not %
            ).name.suffix("_1819_to_2223_pct_change"),
            pl.col(
                # "PEDESTRIAN_ACCIDENT_per_sqmi",
                # "BICYCLE_ACCIDENT_per_sqmi",
                # "ALL_ACCIDENT_per_sqmi",
                "PED_AND_BIKE_ACCIDENT_per_sqmi"
            )
            .diff()
            .name.suffix("_1819_to_2223_diff"),
        )
        # only select every 2nd row (i.e. the post-covid rows)
        .filter(pl.col("covid_period") == "2022-2023")
        .drop(
            "covid_period",
            "PEDESTRIAN_ACCIDENT",
            "BICYCLE_ACCIDENT",
            "ALL_ACCIDENT",
            "PED_AND_BIKE_ACCIDENT",
            "PEDESTRIAN_ACCIDENT_per_sqmi",
            "BICYCLE_ACCIDENT_per_sqmi",
            "ALL_ACCIDENT_per_sqmi",
            "PED_AND_BIKE_ACCIDENT_per_sqmi",
            strict=False,
        )
    )


northeast_core_analysis_neighborhoods = {
    "Financial District/South Beach",
    "Mission Bay",
    "South of Market",
    "Tenderloin",
    "Nob Hill",
    "Chinatown",
    "North Beach",
    "Russian Hill",
}


def add_analysis_neighborhood_geometry(df, analysis_neighborhoods):
    """HOTFIX since I'm doing non spatial calculations in polars"""
    df = analysis_neighborhoods[["analysis_neighborhood", "geometry"]].merge(
        df.to_pandas(), how="right", on="analysis_neighborhood"
    )
    df["northeast_core"] = df["analysis_neighborhood"].isin(
        northeast_core_analysis_neighborhoods
    )
    return df


def plot(
    nhood_crashes,
    accident_col,
    save_filename_stem,
    suptitle=None,
    vmin=None,
    vmax=None,
):
    _, ax = plt.subplots(3, figsize=(10, 10))
    for i, severity in enumerate(["injury only", "fatal", "combined"]):
        # no longer showing ped/bike/all accidents separately
        # for j, accident_col in enumerate(accident_cols):
        nhood_crashes.loc[nhood_crashes["severity"] == severity].plot(
            ax=ax[i], column=accident_col, legend=True, vmin=vmin, vmax=vmax
        )
    plt.suptitle(suptitle)
    plt.savefig(f"output/Links/{save_filename_stem}.png")
    plt.tight_layout()
    plt.show()

In [ ]:
# TODO see if the data in "Q:\Data\Observed\Streets\Safety\SWITRS\San Francisco Data from TIMS" works for this
tims_crashes_filepath = r"C:\Users\cchow\Desktop\tmp\downtown_today\safety\TIMS-SWITRS-crashes-2014-2023.csv"
# crashes only include crashes where injuries/deaths happened
crashes = (
    pl.read_csv(
        tims_crashes_filepath,
        columns=[
            "ACCIDENT_YEAR",
            # "COLLISION_DATE",
            # "COLLISION_TIME",  # some times, e.g. "43" or "20" is unparseable
            "DAY_OF_WEEK",
            "NUMBER_KILLED",
            "NUMBER_INJURED",
            "PEDESTRIAN_ACCIDENT",
            "BICYCLE_ACCIDENT",
            "LATITUDE",
            "LONGITUDE",
            "POINT_X",
            "POINT_Y",
        ],
        schema_overrides={
            "COLLISION_TIME": str,
            "NUMBER_KILLED": int,
            "NUMBER_INJURED": int,
        },
    )
    .filter((pl.col("NUMBER_INJURED") > 0) | (pl.col("NUMBER_KILLED") > 0))
    .select(
        "ACCIDENT_YEAR",
        "DAY_OF_WEEK",
        (pl.col("PEDESTRIAN_ACCIDENT") == "Y").fill_null(False),  # cast to bool
        (pl.col("BICYCLE_ACCIDENT") == "Y").fill_null(False),  # cast to bool
        severity=(
            pl.when(pl.col("NUMBER_KILLED") > 0)
            .then(pl.lit("fatal"))
            .otherwise(pl.lit("injury only"))
        ),
        LATITUDE=pl.when(pl.col("LATITUDE").is_not_null())
        .then(pl.col("LATITUDE"))
        .otherwise(pl.col("POINT_Y")),
        LONGITUDE=pl.when(pl.col("LONGITUDE").is_not_null())
        .then(pl.col("LONGITUDE"))
        .otherwise(pl.col("POINT_X")),
    )
)

analysis_neighborhoods = gpd.read_file(
    r"Q:\GIS\Policy\San_Francisco\Analysis_Neighborhoods\Analysis Neighborhoods_20240610.zip"
).rename(columns={"nhood": "analysis_neighborhood"})
analysis_neighborhoods["area_sqmi"] = (
    analysis_neighborhoods.to_crs("EPSG:7132").area / 27878400  # 7132: SF CRS (ft)
)
# or we can do accidents per street mile:
# street_centrelines = gpd.read_file(
#     r"Q:\GIS\Transportation\Roads\San_Francisco\Street_Centerlines\2022_clean"
# )

crashes = (
    gpd.GeoDataFrame(
        crashes.to_pandas(),
        geometry=gpd.points_from_xy(crashes["LONGITUDE"], crashes["LATITUDE"]),
        crs="EPSG:4326",
    )
    .sjoin(analysis_neighborhoods, how="left", predicate="within")
    .drop(columns="index_right")
)

In [ ]:
# crashes.plot(column="analysis_neighborhood")

In [ ]:
# excludes collisions that are e.g. on bridges within SF County but not within any
# analysis neighborhood. My guess is those should NOT be ped/bike collisions anyways
# the following code plots these collisions not in any analysis neighborhood:
# crashes.loc[crashes["analysis_neighborhood"].isnull(), "analysis_neighborhood"] = "N/A"
# crashes[
#     (crashes["LONGITUDE"] > -123)
#     & (crashes["LONGITUDE"] < -122)
#     & (crashes["LATITUDE"] > 37.5)
#     & (crashes["LATITUDE"] < 38)
# ].plot(column="analysis_neighborhood")

In [ ]:
# HOTFIX cast to pl then back to gpd as I'm more comfortable with pl syntax
crashes_grouped = sum_ped_and_bike(
    group_crashes(  # group into analysis neighborhoods
        # HOTFIX remove geometry col to cast to polars
        pl.from_pandas(crashes.drop(columns="geometry"))
        .with_columns(
            # group years into 2 year periods around COVID
            # (doing 2 year periods to get better statistics with the larger sums)
            covid_period=pl.when(pl.col("ACCIDENT_YEAR").is_in([2018, 2019]))
            .then(pl.lit("2018-2019"))
            .when(pl.col("ACCIDENT_YEAR").is_in([2020, 2021]))
            .then(pl.lit("2020-2021"))
            .when(pl.col("ACCIDENT_YEAR").is_in([2022, 2023]))
            .then(pl.lit("2022-2023"))
        )
        # drop the years not around COVID, and locations not in analysis neighborhoods
        .drop_nulls(),
        "covid_period",
    )
)

# nhood = shorthand for analysis neighborhood
crashes_nhood_change = calculate_covid_change(crashes_grouped, "analysis_neighborhood")
# HOTFIX add back geometry column
crashes_grouped = add_analysis_neighborhood_geometry(
    crashes_grouped, analysis_neighborhoods
)
crashes_nhood_change = add_analysis_neighborhood_geometry(
    crashes_nhood_change, analysis_neighborhoods
)

In [ ]:
crashes_ne_core_grouped = pl.from_pandas(
    crashes_grouped.groupby(["northeast_core", "severity", "covid_period"])
    .agg({"area_sqmi": "sum", "PED_AND_BIKE_ACCIDENT": "sum"})
    .assign(
        PED_AND_BIKE_ACCIDENT_per_sqmi=lambda df: df.PED_AND_BIKE_ACCIDENT
        / df.area_sqmi
    ),
    include_index=True,
)
crashes_ne_core_change = calculate_covid_change(
    crashes_ne_core_grouped, "northeast_core"
)


In [ ]:
crashes_grouped.to_file("output/data/crashes.gpkg")
crashes_nhood_change.to_file("output/data/crashes-change.gpkg")
crashes_ne_core_grouped.write_csv("output/data/crashes-northeast_core.csv")
crashes_ne_core_change.write_csv("output/data/crashes-northeast_core-change.csv")


In [ ]:
ne_core_pct_change_chart = (
    crashes_ne_core_change.filter(pl.col("severity").is_in(["injury only", "fatal"]))
    .plot.bar(
        x="severity",
        y="PED_AND_BIKE_ACCIDENT_1819_to_2223_pct_change",
        color="northeast_core",
        xOffset="northeast_core",
    )
    .properties(title="pct change in ped+bike collisions 18/19 vs 22/23")
)
ne_core_pct_change_chart.save("output/Links/crashes-by_northeast_core-change.png")
ne_core_pct_change_chart


In [ ]:
ne_core_chart = (
    alt.Chart(
        crashes_ne_core_grouped.filter(
            pl.col("covid_period").is_in(["2018-2019", "2022-2023"])
            & pl.col("severity").is_in(["injury only"])
        )
    )
    .mark_bar()
    .encode(
        x=alt.X("covid_period:N"),
        xOffset="northeast_core:N",
        y=alt.Y("PED_AND_BIKE_ACCIDENT_per_sqmi:Q"),
        color=alt.Color("northeast_core:N"),
    )
    .properties(title="ped & bike accidents per sqmi (injury only)")
) | (
    alt.Chart(
        crashes_ne_core_grouped.filter(
            pl.col("covid_period").is_in(["2018-2019", "2022-2023"])
            & pl.col("severity").is_in(["fatal"])
        )
    )
    .mark_bar()
    .encode(
        x=alt.X("covid_period:N"),
        xOffset="northeast_core:N",
        y=alt.Y("PED_AND_BIKE_ACCIDENT_per_sqmi:Q"),
        color=alt.Color("northeast_core:N"),
    )
    .properties(title="ped & bike accidents per sqmi (fatal)")
)  # .resolve_scale(y="shared")
ne_core_chart.save("output/Links/crashes-by_northeast_core.png")
ne_core_chart


In [ ]:
_, ax = plt.subplots(3, 3, figsize=(10, 10))
for i, (severity, vmax) in enumerate(
    zip(["injury only", "fatal", "combined"], [600, 16, 650])
):
    for j, covid_period in enumerate(["2018-2019", "2020-2021", "2022-2023"]):
        crashes_grouped.loc[
            (crashes_grouped["severity"] == severity)
            & (crashes_grouped["covid_period"] == covid_period)
        ].plot(
            ax=ax[i, j],
            column="PED_AND_BIKE_ACCIDENT_per_sqmi",
            legend=True,
            vmin=0,
            vmax=vmax,
        )
plt.suptitle(
    "ped+bike crashes per sqmi: 18/19 vs 20/21 vs 22/23\n"
    "rows = injury only, fatal, combined\ncols = 18/19, 20/21, 22/23\n"
    "holes = NaN = 0 to 0"
)
plt.savefig("output/Links/crashes-by_analysis_neighborhood.png")
plt.tight_layout()
plt.show()

In [ ]:
_, ax = plt.subplots(2, 2, figsize=(10, 10))
for i, severity in enumerate(
    [
        "injury only",
        # "fatal",
        "combined",
    ]
):
    for j, (col, vminmax) in enumerate(
        zip(
            [
                "PED_AND_BIKE_ACCIDENT_per_sqmi_1819_to_2223_diff",
                "PED_AND_BIKE_ACCIDENT_1819_to_2223_pct_change",
            ],
            [180, 150],
        )
    ):
        crashes_nhood_change.loc[crashes_nhood_change["severity"] == severity].plot(
            ax=ax[i, j],
            column=col,
            legend=True,
            vmin=-vminmax,
            vmax=vminmax,
            cmap="coolwarm",
        )
plt.suptitle(
    "ped+bike crashes: 18/19 vs 22/23\n"
    "rows = injury only, combined\ncols = diff, pct change\n"
    "holes = NaN = '0 to 0'"
)
plt.savefig("output/Links/crashes-by_analysis_neighborhood-1819v2223.png")
plt.tight_layout()
plt.show()